# Perform Statistical Analysis for AMR genes

This analysis is based on: https://www.kaggle.com/code/hamelg/python-for-data-25-chi-squared-tests

## Get high risk patients

### Define a function to read data from the FHIR server

In [1]:
import json
import requests


def get(url):
    response = requests.get(
        url=url,
        headers={"Content-Type": "application/fhir+json", "authentication": "mjRmoNGW6klxaClkKhEkqi7HVYwx6NTH"},
    )
    return response


def readData(url):
    nextUrl = url
    data = []
    while(nextUrl):
        print('Reading URL: ', nextUrl)
        response = get(nextUrl)
        responseText = json.loads(response.text)
        data.append(responseText)
        nextUrl = None
        if 'link' in responseText:
            for link in responseText['link']:
                if link['relation'] == 'next':
                    nextUrl = link['url']
    return data

### Obtain high risk patient ids from FHIR

In [11]:
import itertools

[lowerRiskScore, higherRiskScore] = [0.5, 1.0]
query = 'http://10.172.235.4:8080/fhir/Patient?_has:RiskAssessment:subject:probability=ge' + str(lowerRiskScore) + '&_has:RiskAssessment:subject:probability=le' + str(higherRiskScore)
response = readData(query)

highriskPatientIds = list(itertools.chain.from_iterable(list(map(lambda data: list(map(lambda entry: entry['resource']['id'][1:], data['entry'])), response))))
highriskPatientIds

Reading URL:  http://10.172.235.4:8080/fhir/Patient?_has:RiskAssessment:subject:probability=ge0.5&_has:RiskAssessment:subject:probability=le1.0
Reading URL:  http://10.172.235.4:8080/fhir?_getpages=0f45a4f1-b5f4-4107-9321-e4d486645d92&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset


['2202086',
 '2172200',
 '546011',
 '1013210',
 '2103171',
 '2013664',
 '2120861',
 '2142899',
 '677694',
 '2448944',
 '2150228',
 '745962',
 '2153643',
 '1581023',
 '2140940',
 '631550',
 '2198313',
 '2161817',
 '2107492',
 '2141593',
 '549608',
 '2160210',
 '637422',
 '2092580',
 '2156000',
 '2199705']

In [12]:
len(highriskPatientIds)

26

## Map tube codes for high risk patients

In [13]:
import os

import pandas as pd


mappingDf = pd.read_csv(os.environ['GENOMICS_DATA_BASE'] + '/index_saur.csv')
mappingDf

,specimen_id,patient_id,remap_file,amr_file,fasta_info_file,remap_info_file
0,AH18J001,2145906,/home/vmadmin/workspace/genome_data/remap_file...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...
1,AH18J002,2154958,/home/vmadmin/workspace/genome_data/remap_file...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...
2,AH18J014,652485,/home/vmadmin/workspace/genome_data/remap_file...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...
3,AH18J021,2054932,/home/vmadmin/workspace/genome_data/remap_file...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...
4,AH18J022,2131734,/home/vmadmin/workspace/genome_data/remap_file...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...
...,...,...,...,...,...,...
362,AH21H055,297900,/home/vmadmin/workspace/genome_data/remap_file...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...
363,AH21H060,745862,/home/vmadmin/workspace/genome_data/remap_file...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...
364,AH21H068,2526402,/home/vmadmin/workspace/genome_data/remap_file...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...
365,AH21H076,2180362,/home/vmadmin/workspace/genome_data/remap_file...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...,https://raw.githubusercontent.com/ryashpal/ehr...


In [14]:
highriskTubecodes = list(mappingDf[mappingDf.patient_id.isin([int(patientId) for patientId in highriskPatientIds])].specimen_id)
highriskTubecodes

['AH18J055',
 'AH18J081',
 'AH18K024',
 'AH19A043',
 'AH19B014',
 'AH19B026',
 'AH19C012',
 'AH19C070',
 'AH19E069',
 'AH19F055',
 'AH19J028',
 'AH19K013',
 'AH19L077',
 'AH20C005',
 'AH20E051',
 'AH20F012',
 'AH20G044',
 'AH20H023',
 'AH20I042',
 'AH20I057',
 'AH20K063',
 'AH20L008',
 'AH20L085',
 'AH21A034',
 'AH21A036',
 'AH21C022']

## Read annotations

In [15]:
import os

import pandas as pd


highriskAnnotationsDfList = []
controlAnnotationsDfList = []

gffDir = os.environ['GENOMICS_DATA_BASE'] + '/amrfinder'

for fileName in os.listdir(gffDir):

    tubeCode = fileName.split('.')[0].split('_')[0]

    amrResultsDf = pd.read_csv(
        gffDir + '/' + fileName,
        sep='\t',
    )
    amrResultsDf['tube_code'] = tubeCode

    if tubeCode in highriskTubecodes:
        highriskAnnotationsDfList.append(amrResultsDf)
    else:
        controlAnnotationsDfList.append(amrResultsDf)

highriskAnnotationsDf = pd.concat(highriskAnnotationsDfList, ignore_index=True)
controlAnnotationsDf = pd.concat(controlAnnotationsDfList, ignore_index=True)

highriskAnnotationsDf.shape, controlAnnotationsDf.shape

((595, 23), (57020, 23))

In [16]:
len(highriskAnnotationsDf.tube_code.unique()), len(controlAnnotationsDf.tube_code.unique())

(26, 2954)

## Perform chi-square tests

In [17]:
controlAnnotationsDf[['Element type', 'Element subtype']].drop_duplicates()

,Element type,Element subtype
0,VIRULENCE,VIRULENCE
3,AMR,AMR
7,STRESS,BIOCIDE
18,STRESS,METAL
29,STRESS,ACID
220,STRESS,HEAT


In [35]:
import scipy.stats as stats


dfDict = {}

for annotationType in ['VIRULENCE', 'AMR', 'STRESS']:
# for annotationType in ['AMR']:

    print('annotationType: ', annotationType)

    highriskGenecountsDf = highriskAnnotationsDf[highriskAnnotationsDf['Element type'] == annotationType][['Contig id', 'Gene symbol']].groupby(
            by=['Gene symbol']
        ).agg(
            'count'
        ).reset_index().rename(columns={'Contig id': 'high_risk_genes_count'})

    controlGenecountsDf = controlAnnotationsDf[controlAnnotationsDf['Element type'] == annotationType][['Contig id', 'Gene symbol']].groupby(
            by=['Gene symbol']
        ).agg(
            'count'
        ).reset_index().rename(columns={'Contig id': 'control_genes_count'})

    mergedGenecountsDf = controlGenecountsDf.merge(
        highriskGenecountsDf,
        how='left',
        on=['Gene symbol']
    ).fillna(0)

    mergedGenecountsDf['control_genes_proportion'] = mergedGenecountsDf.control_genes_count/mergedGenecountsDf.control_genes_count.sum()

    mergedGenecountsDf['expected_genes_count'] = mergedGenecountsDf.control_genes_proportion * mergedGenecountsDf.high_risk_genes_count.sum()

    filteredGenecountsDf = mergedGenecountsDf[(mergedGenecountsDf.high_risk_genes_count >= 5) & (mergedGenecountsDf.expected_genes_count >= 5) & (mergedGenecountsDf.high_risk_genes_count > 0)]

    dfDict[annotationType] = filteredGenecountsDf

    if(filteredGenecountsDf.shape[0] < 2):
        print('Not sufficient data for the test')
        continue

    chi2, p, dof, expected = stats.chi2_contingency(pd.crosstab(filteredGenecountsDf.high_risk_genes_count, filteredGenecountsDf.expected_genes_count), correction=True)
    significant = p < 0.05  # 5% significance level
    print(chi2, p, significant)

annotationType:  VIRULENCE
55.0 0.05745735167659182 False
annotationType:  AMR
Not sufficient data for the test
annotationType:  STRESS
Not sufficient data for the test


In [41]:
import scipy.stats as stats


dfDict = {}

for annotationType in ['VIRULENCE', 'AMR', 'STRESS']:
# for annotationType in ['AMR']:

    print('annotationType: ', annotationType)

    highriskGenecountsDf = highriskAnnotationsDf[highriskAnnotationsDf['Element type'] == annotationType][['Contig id', 'Gene symbol']].groupby(
            by=['Gene symbol']
        ).agg(
            'count'
        ).reset_index().rename(columns={'Contig id': 'high_risk_genes_count'})

    controlGenecountsDf = controlAnnotationsDf[controlAnnotationsDf['Element type'] == annotationType][['Contig id', 'Gene symbol']].groupby(
            by=['Gene symbol']
        ).agg(
            'count'
        ).reset_index().rename(columns={'Contig id': 'control_genes_count'})

    mergedGenecountsDf = controlGenecountsDf.merge(
        highriskGenecountsDf,
        how='left',
        on=['Gene symbol']
    ).fillna(0)

    filteredGenecountsDf = mergedGenecountsDf[(mergedGenecountsDf.high_risk_genes_count >= 5) & (mergedGenecountsDf.high_risk_genes_count > 0)]

    filteredGenecountsDf['control_genes_proportion'] = filteredGenecountsDf.control_genes_count/filteredGenecountsDf.control_genes_count.sum()

    filteredGenecountsDf['expected_genes_count'] = filteredGenecountsDf.control_genes_proportion * filteredGenecountsDf.high_risk_genes_count.sum()

    dfDict[annotationType] = filteredGenecountsDf

    if(filteredGenecountsDf.shape[0] < 2):
        print('Not sufficient data for the test')
        continue

    chi2, p = stats.chisquare(f_obs=filteredGenecountsDf.high_risk_genes_count, f_exp=filteredGenecountsDf.expected_genes_count)
    significant = p < 0.05  # 5% significance level
    print(chi2, p, significant)

annotationType:  VIRULENCE
10.869132557035037 0.9495459165974602 False
annotationType:  AMR
1.399489055858899 0.9658894901013364 False
annotationType:  STRESS
46.65687621578695 1.7976381917800828e-09 True


/tmp/ipykernel_1024302/277457695.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filteredGenecountsDf['control_genes_proportion'] = filteredGenecountsDf.control_genes_count/filteredGenecountsDf.control_genes_count.sum()
/tmp/ipykernel_1024302/277457695.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filteredGenecountsDf['expected_genes_count'] = filteredGenecountsDf.control_genes_proportion * filteredGenecountsDf.high_risk_genes_count.sum()
/tmp/ipykernel_1024302/277457695.py:31: SettingWithCopyWa

In [ ]:
dfDict['VIRULENCE']

,Gene symbol,control_genes_count,high_risk_genes_count,control_genes_proportion,expected_genes_count
16,aur,535,26.0,0.073038,25.490102
44,hld,535,26.0,0.073038,25.490102
45,hlgA,531,26.0,0.072491,25.299522
46,hlgB,531,26.0,0.072491,25.299522
47,hlgC,530,26.0,0.072355,25.251877
51,icaC,536,27.0,0.073174,25.537747
68,lukD,244,8.0,0.033311,11.625392
69,lukE,353,13.0,0.048191,16.818703
91,sak,417,23.0,0.056928,19.867986
93,scn,514,24.0,0.070171,24.489556


In [42]:
dfDict['AMR']

,Gene symbol,control_genes_count,high_risk_genes_count,control_genes_proportion,expected_genes_count
109,blaI,451,21.0,0.173863,21.211257
244,blaR1,313,14.0,0.120663,14.720894
288,blaZ,379,17.0,0.146106,17.824981
339,fosB,296,12.0,0.114109,13.921357
357,mecA,85,6.0,0.032768,3.997687
363,mepA,535,26.0,0.206245,25.161912
420,tet(38),535,26.0,0.206245,25.161912


In [43]:
dfDict['STRESS']

,Gene symbol,control_genes_count,high_risk_genes_count,control_genes_proportion,expected_genes_count
1,arsB,284,6.0,0.102122,5.923049
2,arsC,1233,6.0,0.443366,25.715210
6,arsR,438,6.0,0.157497,9.134844
12,cadD,335,15.0,0.120460,6.986695
25,lmrS,491,25.0,0.176555,10.240201


## Old Code

In [20]:
filteredGenecountsDf.high_risk_genes_count.sum() / filteredGenecountsDf.expected_genes_count.sum() * filteredGenecountsDf.expected_genes_count

2     2.756334
21    3.243666
Name: expected_genes_count, dtype: float64

In [ ]:
mergedGenecountsDf['percentage_difference'] = (mergedGenecountsDf.expected_genes_count - mergedGenecountsDf.high_risk_genes_count)/mergedGenecountsDf.expected_genes_count * 100
mergedGenecountsDf

,Gene symbol,control_genes_count,high_risk_genes_count,control_genes_proportion,expected_genes_count,percentage_difference
0,aac(3)-IId,76,0.0,0.004225,0.612665,100.0
1,aac(3)-IIe,19,0.0,0.001056,0.153166,100.0
2,aac(3)-IIg,1,0.0,0.000056,0.008061,100.0
3,aac(3)-IVa,1,0.0,0.000056,0.008061,100.0
4,aac(6'),36,0.0,0.002001,0.290210,100.0
...,...,...,...,...,...,...
453,vanX-B,135,0.0,0.007505,1.088286,100.0
454,vanXY-C,8,0.0,0.000445,0.064491,100.0
455,vanY-A,20,0.0,0.001112,0.161228,100.0
456,vanY-B,134,0.0,0.007450,1.080225,100.0


In [ ]:
mergedGenecountsDf[mergedGenecountsDf.high_risk_genes_count == 0]

,Gene symbol,control_genes_count,high_risk_genes_count,control_genes_proportion,expected_genes_count,percentage_difference
0,aac(3)-IId,76,0.0,0.004225,0.612665,100.0
1,aac(3)-IIe,19,0.0,0.001056,0.153166,100.0
2,aac(3)-IIg,1,0.0,0.000056,0.008061,100.0
3,aac(3)-IVa,1,0.0,0.000056,0.008061,100.0
4,aac(6'),36,0.0,0.002001,0.290210,100.0
...,...,...,...,...,...,...
453,vanX-B,135,0.0,0.007505,1.088286,100.0
454,vanXY-C,8,0.0,0.000445,0.064491,100.0
455,vanY-A,20,0.0,0.001112,0.161228,100.0
456,vanY-B,134,0.0,0.007450,1.080225,100.0


In [ ]:
mergedGenecountsDf[mergedGenecountsDf.high_risk_genes_count != 0].sort_values(by=['percentage_difference'])

,Gene symbol,control_genes_count,high_risk_genes_count,control_genes_proportion,expected_genes_count,percentage_difference
341,fusC,12,2.0,0.000667,0.096737,-1967.471264
428,tet(K),19,2.0,0.001056,0.153166,-1205.771325
357,mecA,85,6.0,0.004726,0.685217,-775.634888
207,blaPC1,65,4.0,0.003614,0.523990,-663.374005
363,mepA,535,26.0,0.029744,4.312837,-502.851434
420,tet(38),535,26.0,0.029744,4.312837,-502.851434
359,mecR1,64,3.0,0.003558,0.515928,-481.476293
109,blaI,451,21.0,0.025074,3.635681,-477.608380
288,blaZ,379,17.0,0.021071,3.055262,-456.417069
244,blaR1,313,14.0,0.017401,2.523211,-454.848518
